In [4]:
import pandas as pd
import json
import numpy as np
from dtw import dtw

In [5]:
#algoritmo principale
def DTWDistance(signalA, signalB, channel):
    n,m = len(signalA), len(signalB)
    DTW = np.zeros(shape=(n, m))
    for i in range(0,n):
        DTW[i,0] = np.finfo(dtype=float).max
    for i in range(0,m):
        DTW[0,i] = np.finfo(dtype=float).max
    DTW[0,0] = 0
    
    for i in range(1,n):
        for j in range(1,m):
            const = abs(signalA[i][channel] - signalB[j][channel])
            DTW[i,j] = const + min(DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1])
    
    return DTW[n - 1, m - 1]

In [6]:
#Algoritmo di scaling
def new_scale_df(data_frame, gyro, accl, pres):
    
    def scale(value, max_, min_):
        return (value - min_) / (max_ - min_)
        
    copy = data_frame.copy() 
    
    for wIndex in range(len(copy)):
        for vIndex in range(len(copy[wIndex])):
            copy[wIndex][vIndex][0] = scale(copy[wIndex][vIndex][0], accl, -accl)
            copy[wIndex][vIndex][1] = scale(copy[wIndex][vIndex][1], accl, -accl)
            copy[wIndex][vIndex][2] = scale(copy[wIndex][vIndex][2], accl, -accl)
            copy[wIndex][vIndex][3] = scale(copy[wIndex][vIndex][3], pres, 0)
            copy[wIndex][vIndex][4] = scale(copy[wIndex][vIndex][4], gyro, -gyro)
            copy[wIndex][vIndex][5] = scale(copy[wIndex][vIndex][5], gyro, -gyro)
            copy[wIndex][vIndex][6] = scale(copy[wIndex][vIndex][6], gyro, -gyro)
    
    return copy

In [7]:
with open('./giovanni_new.json', 'r') as f:
    data_giovanni = json.load(f)
with open('./dario_new.json', 'r') as f:
    data_dario = json.load(f)

In [8]:
scaled_giovanni = new_scale_df(data_giovanni, 250, 2, 1024)
scaled_dario = new_scale_df(data_dario, 250, 2, 1024)

In [17]:
#for i in range(7):
#    print('indice: ', i)
#    print('gio vs dario',DTWDistance(scaled_giovanni[1], scaled_dario[1],i))
#    print('gio vs dario',DTWDistance(scaled_giovanni[2], scaled_dario[2],i))
#    print('gio vs gio',DTWDistance(scaled_giovanni[1], scaled_giovanni[2],i))
#    print('dario vs dario', DTWDistance(scaled_dario[1], scaled_dario[2],i))
l2_norm = lambda x, y: (x[0] - y[0]) ** 2
d1, cost_matrix, acc_cost_matrix, path = dtw(scaled_giovanni[1], scaled_dario[1], dist=l2_norm)
d2, cost_matrix, acc_cost_matrix, path = dtw(scaled_giovanni[1], scaled_giovanni[2], dist=l2_norm)
print(d1, d2)

0.001393935575290482 0.0008340617796671061
